In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fractions import Fraction as frac
from decimal import Decimal
import regex as re

In [2]:
url = 'https://maplestory.nexon.com/Guide/OtherProbability/cube/red'


In [26]:
def process_df(df,first=True,firstweights=None,primeprob=None):
    split = df.iloc[:,0].str.split(':',expand=True)
    split.columns = ['Stat','Value']
    notcooldowns = split.Stat.apply(lambda x: "".join(x.split())) != '모든스킬의재사용대기시간'
    percentlines = split.Value.str.contains('%',na=False) & notcooldowns
    split.Stat.loc[percentlines] = split.Stat.loc[percentlines] + "%"
    print(split.Stat.loc[percentlines])
    split.Value = split.Value.str.extract('([0-9]+)').astype('int',errors='ignore')
    split.fillna({'Value':1},inplace=True)
    split.Stat = split.Stat.str.strip()
    probs = df.iloc[:,1].apply(lambda x: float(x.strip('%'))/100)
    if first:
        reduced = probs/min(probs)
        weights = reduced.apply(lambda x: frac(Decimal(x)).limit_denominator(100) * max(reduced.apply(lambda x: frac(Decimal(x)).limit_denominator(100).denominator))).astype('int')
    else:
        weights = pd.Series(index=probs.index)
        weights[-len(firstweights):] = firstweights
        weights[:-len(firstweights)] = probs[:-len(firstweights)] / primeprob * sum(firstweights)
        weights = weights.apply(lambda x: frac(Decimal(x)).limit_denominator(20) * max(weights.apply(lambda x: frac(Decimal(x)).limit_denominator(20).denominator)))
    split['Weight'] = weights
    return split



In [20]:
def get_cube_weights(url):
    html = requests.get(url).content
    df_list = pd.read_html(html, match='STR')
    soup = BeautifulSoup(html, 'html.parser')
    table_properties = soup.find_all('div',{'class': 'cube_option'})
    cube_df = []
    for i in range(0,len(df_list)):
        metadata = re.findall('<span>(.+)</span>',str(table_properties[i]))
        if metadata[0] == '레어':
            no_rare_lines = len(df_list[i].iloc[:,0:2].dropna())
            normal_lines = df_list[i].iloc[:-no_rare_lines,2:4].dropna()
            minidf = process_df(normal_lines)
            minidf['Grade'] = 'Normal'
            minidf['Equip'] = metadata[1]
            minidf['Level Requirement'] = int(re.findall('([0-9]+)',metadata[2])[0])
            minidf = minidf[['Equip','Level Requirement','Grade','Stat','Value','Weight']]
            cube_df.append(minidf)
        primelines = df_list[i].iloc[:,0:2].dropna()
        minidf = process_df(primelines)
        minidf['Grade'] = metadata[0]
        minidf['Equip'] = metadata[1]
        minidf['Level Requirement'] = int(re.findall('([0-9]+)',metadata[2])[0])
        minidf = minidf[['Equip','Level Requirement','Grade','Stat','Value','Weight']]
        cube_df.append(minidf)

    alldf = pd.concat(cube_df,ignore_index=True)
    return alldf

In [27]:
cubedict = {'Red':'https://maplestory.nexon.com/Guide/OtherProbability/cube/red',
           'Black': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/black',
           'Additional': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/addi',
           'Suspicious': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/strange',
           'Master Craftsman': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/master',
           'Meister': 'https://maplestory.nexon.com/Guide/OtherProbability/cube/artisan'}
df = []
for cube,url in cubedict.items():
    cubeweights = get_cube_weights(url)
    cubeweights['Cube'] = cube
    df.append(cubeweights)

masterlist = pd.concat(df,ignore_index=True)
masterlist.to_csv('weights_kr.csv',index=False)

Series([], Name: Stat, dtype: object)
8            STR %
9            DEX %
10           INT %
11           LUK %
12           공격력 %
13            마력 %
14       크리티컬 확률 %
15           데미지 %
25    몬스터 방어율 무시 %
Name: Stat, dtype: object
0            STR %
1            DEX %
2            INT %
3            LUK %
4          최대 HP %
5          최대 MP %
6            공격력 %
7             마력 %
8        크리티컬 확률 %
9            데미지 %
10           올스탯 %
13    몬스터 방어율 무시 %
Name: Stat, dtype: object
0                 STR %
1                 DEX %
2                 INT %
3                 LUK %
4                 공격력 %
5                  마력 %
6             크리티컬 확률 %
7                 데미지 %
8                 올스탯 %
9          몬스터 방어율 무시 %
10    보스 몬스터 공격 시 데미지 %
11    보스 몬스터 공격 시 데미지 %
Name: Stat, dtype: object
0                 STR %
1                 DEX %
2                 INT %
3                 LUK %
4                 공격력 %
5                  마력 %
6             크리티컬 확률 %
7                 데미지 %
8    

7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
0                      STR %
1                      DEX %
2                      INT %
3                      LUK %
4                    최대 HP %
5                    최대 MP %
6                      방어력 %
7                      올스탯 %
10    HP 회복 아이템 및 회복 스킬 효율 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
0                      STR %
1       

Name: Stat, dtype: object
0                 STR %
1                 DEX %
2                 INT %
3                 LUK %
4                 공격력 %
5                  마력 %
6             크리티컬 확률 %
7                 데미지 %
8                 올스탯 %
9          몬스터 방어율 무시 %
12    보스 몬스터 공격 시 데미지 %
13    보스 몬스터 공격 시 데미지 %
Name: Stat, dtype: object
0                 STR %
1                 DEX %
2                 INT %
3                 LUK %
4                 공격력 %
5                  마력 %
6             크리티컬 확률 %
7                 데미지 %
8                 올스탯 %
11         몬스터 방어율 무시 %
12         몬스터 방어율 무시 %
15    보스 몬스터 공격 시 데미지 %
16    보스 몬스터 공격 시 데미지 %
17    보스 몬스터 공격 시 데미지 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
8            STR %
9            DEX %
10           INT %
11           LUK %
12           공격력 %
13            마력 %
14       크리티컬 확률 %
15           데미지 %
25    몬스터 방어율 무시 %
Name: Stat, dtype: object
0            STR %
1            DEX %
2            INT %
3     

Name: Stat, dtype: object
0                     STR %
1                     DEX %
2                     INT %
3                     LUK %
4                   최대 HP %
5                   최대 MP %
6                     방어력 %
7                     올스탯 %
8    HP 회복 아이템 및 회복 스킬 효율 %
Name: Stat, dtype: object
0              STR %
1              DEX %
2              INT %
3              LUK %
4            최대 HP %
5            최대 MP %
6              방어력 %
7              올스탯 %
8     모든 스킬의 MP 소모 %
9     모든 스킬의 MP 소모 %
10          메소 획득량 %
11         아이템 드롭률 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
0                     STR %
1                     DEX %
2                     INT %
3                     LUK %
4             

11      STR %
12      DEX %
13      INT %
14      LUK %
15    최대 HP %
16    최대 MP %
17      방어력 %
18      올스탯 %
Name: Stat, dtype: object
9                      STR %
10                     DEX %
11                     INT %
12                     LUK %
13                   최대 HP %
14                   최대 MP %
15                     방어력 %
16                     올스탯 %
21                모든 속성 내성 %
22    HP 회복 아이템 및 회복 스킬 효율 %
Name: Stat, dtype: object
9                      STR %
10                     DEX %
11                     INT %
12                     LUK %
13                   최대 HP %
14                   최대 MP %
15                     방어력 %
16                크리티컬 데미지 %
17                크리티컬 데미지 %
18                     올스탯 %
23                모든 속성 내성 %
24    HP 회복 아이템 및 회복 스킬 효율 %
25                  메소 획득량 %
26                 아이템 드롭률 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
11      STR %
12      DEX %
13      INT %
14      LUK %
15    최대 HP %
16    

11      STR %
12      DEX %
13      INT %
14      LUK %
15    최대 HP %
16    최대 MP %
17      방어력 %
18      올스탯 %
Name: Stat, dtype: object
9                      STR %
10                     DEX %
11                     INT %
12                     LUK %
13                   최대 HP %
14                   최대 MP %
15                     방어력 %
16                     올스탯 %
21                모든 속성 내성 %
22    HP 회복 아이템 및 회복 스킬 효율 %
Name: Stat, dtype: object
9                      STR %
10                     DEX %
11                     INT %
12                     LUK %
13                   최대 HP %
14                   최대 MP %
15                     방어력 %
16                     올스탯 %
21                모든 속성 내성 %
22            모든 스킬의 MP 소모 %
23    HP 회복 아이템 및 회복 스킬 효율 %
24                  메소 획득량 %
25                 아이템 드롭률 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
11      STR %
12      DEX %
13      INT %
14      LUK %
15    최대 HP %
16    최대 MP %
17      방어력 %
Name: S

7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object


0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
0                      STR %
1                      DEX %
2                      INT %
3                      LUK %
4                    최대 HP %
5                    최대 MP %
6                      방어력 %
7                      올스탯 %
10    HP 회복 아이템 및 회복 스킬 효율 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
0                      STR %
1                      DEX %
2                      INT %
3                      LUK %
4                    최대 HP %
5                    최대 MP %
6                      방어력 %
7                      올스탯 %
10    HP 회복 아이템 및 회복 스킬 효율 %
Name: Stat, dtype: object
Seri

Name: Stat, dtype: object
0                 STR %
1                 DEX %
2                 INT %
3                 LUK %
4                 공격력 %
5                  마력 %
6             크리티컬 확률 %
7                 데미지 %
8                 올스탯 %
9          몬스터 방어율 무시 %
10         몬스터 방어율 무시 %
13    보스 몬스터 공격 시 데미지 %
14    보스 몬스터 공격 시 데미지 %
15    보스 몬스터 공격 시 데미지 %
Name: Stat, dtype: object
Series([], Name: Stat, dtype: object)
8            STR %
9            DEX %
10           INT %
11           LUK %
12           공격력 %
13            마력 %
14       크리티컬 확률 %
15           데미지 %
25    몬스터 방어율 무시 %
Name: Stat, dtype: object
0            STR %
1            DEX %
2            INT %
3            LUK %
4          최대 HP %
5          최대 MP %
6            공격력 %
7             마력 %
8        크리티컬 확률 %
9            데미지 %
10           올스탯 %
13    몬스터 방어율 무시 %
Name: Stat, dtype: object
0                 STR %
1                 DEX %
2                 INT %
3                 LUK %
4                 공격력 %
5  

Series([], Name: Stat, dtype: object)
7       STR %
8       DEX %
9       INT %
10      LUK %
11    최대 HP %
12    최대 MP %
13      방어력 %
Name: Stat, dtype: object
0      STR %
1      DEX %
2      INT %
3      LUK %
4    최대 HP %
5    최대 MP %
6      방어력 %
7      올스탯 %
Name: Stat, dtype: object
0                     STR %
1                     DEX %
2                     INT %
3                     LUK %
4                   최대 HP %
5                   최대 MP %
6                     방어력 %
7                     올스탯 %
8    HP 회복 아이템 및 회복 스킬 효율 %
Name: Stat, dtype: object
0                      STR %
1                      DEX %
2                      INT %
3                      LUK %
4                    최대 HP %
5                    최대 MP %
6                      방어력 %
7                      올스탯 %
8             모든 스킬의 MP 소모 %
9             모든 스킬의 MP 소모 %
10    HP 회복 아이템 및 회복 스킬 효율 %
11                  메소 획득량 %
12                 아이템 드롭률 %
Name: Stat, dtype: object
Series([], Name: Stat, dtyp

In [28]:
# koreancolumns = masterlist[['Equip','Grade','Stat']]
# koreanwords = []
# for i in koreancolumns.columns:
#     unique = koreancolumns[i].unique()
#     koreanwords += [x for x in unique]
# with open('koreanwords.csv','w+') as f:
#     for j in koreanwords:
#         f.write('"'+j+'"\n')

dictionary = pd.read_excel('translations.xlsx')
def translate(df,fromlang='kr',tolang='en'):
    translation_map = dictionary.set_index(fromlang)[tolang].to_dict()
    df2 = df.copy()
    for i in df.columns:
        df2[i] = df[i].map(lambda x: translation_map.get(x,x))
    return df2

translate(masterlist).to_csv('weights_en.csv',index=False)

In [13]:
masterlist['Stat'].unique()

array(['STR', 'DEX', 'INT', 'LUK', '최대 HP', '최대 MP', '공격력', '마력', 'STR %',
       'DEX %', 'INT %', 'LUK %', '공격력 %', '마력 %', '크리티컬 확률 %', '데미지 %',
       '올스탯', '공격 시 20% 확률로 240의 HP 회복', '공격 시 20% 확률로 120의 MP 회복',
       '공격 시 20% 확률로 6레벨 중독효과 적용', '공격 시 10% 확률로 2레벨 기절효과 적용',
       '공격 시 20% 확률로 2레벨 슬로우효과 적용', '공격 시 20% 확률로 3레벨 암흑효과 적용',
       '공격 시 10% 확률로 2레벨 빙결효과 적용', '공격 시 10% 확률로 2레벨 봉인효과 적용',
       '몬스터 방어율 무시 %', '최대 HP %', '최대 MP %', '올스탯 %',
       '공격 시 20% 확률로 360의 HP 회복', '공격 시 20% 확률로 180의 MP 회복',
       '보스 몬스터 공격 시 데미지 %', '캐릭터 기준 10레벨 당 공격력', '캐릭터 기준 10레벨 당 마력',
       '공격 시 20% 확률로 200의 HP 회복', '공격 시 20% 확률로 110의 MP 회복',
       '공격 시 20% 확률로 5레벨 중독효과 적용', '공격 시 20% 확률로 2레벨 암흑효과 적용',
       '공격 시 20% 확률로 300의 HP 회복', '공격 시 20% 확률로 165의 MP 회복',
       '피격 시 5% 확률로 데미지의 20% 무시', '피격 시 5% 확률로 데미지의 40% 무시',
       '피격 시 10% 확률로 데미지의 20% 무시', '피격 시 10% 확률로 데미지의 40% 무시', '방어력',
       '방어력 %', '모든  스킬레벨', 'HP 회복 아이템 및 회복 스킬 효율 %',
       '<쓸만한 미스틱 도어> 스킬 사용  가능', '모든  스킬